In [55]:
!pip install -q -U kaggle_environments

In [56]:
#import numpy as np
#import pandas as pd
#import random

#import matplotlib.pyplot as plt
#import seaborn as sns

from kaggle_environments import make, evaluate

**Собственные агенты**

1. Всегда "бумага"

In [57]:
%%writefile paper_agent.py
def paper_agent(observation, configuration):
    return 1    

Overwriting paper_agent.py


2. Всегда "ножницы"

In [58]:
%%writefile scissors_agent.py
def scissors_agent(observation, configuration):
    return 2    

Overwriting scissors_agent.py


3. Случайный выбор

In [59]:
%%writefile random_agent.py
import random


def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Overwriting random_agent.py


4. перебор всех вариантов по порядку циклом

In [60]:
%%writefile next_agent.py
import random


my_last_action = None

def next_agent(observation, configuration):
    global my_last_action
    if observation.step == 0:
        action = random.randrange(0, configuration.signs)
    else:
        action = (my_last_action + 1) % configuration.signs
    my_last_action = action
    return action

Overwriting next_agent.py


5. Смотрим, что чаще всего выбирает оппонент и подстраиваемся

In [61]:
%%writefile simple_adaptation_agent.py
import random


opponent_choices = {}


def simple_adaptation_agent(observation, configuration):
    global opponent_choices
    if observation.step == 0:
    # если это первая игра обнуляем статистику и делаем случайный выбор
        opponent_choices = {}
        action = random.randrange(0, configuration.signs)
    else:
        last_opponent_action = observation.lastOpponentAction
        # если это действие еще не встречалось, добавляем его в статистику
        if last_opponent_action not in opponent_choices:
            opponent_choices[last_opponent_action] = 0
        # обновляем статистику
        opponent_choices[last_opponent_action] += 1
        # выбираем из статистики наиболее частое действие противника и выбираем
        # действие с номером +1 от противника т.е. на 0(камень) реагируем
        # 1(бумага)

        act = None
        act_count = None
        for k, v in opponent_choices.items():
            if act is None or v > act_count:
                act = k
                act_count = v
                continue
        action = (act + 1) % configuration.signs
    return action

Overwriting simple_adaptation_agent.py


6. Запоминаем последовательность действий противника (0, 2, 1...) и угадываем следующий его шаг, т.е. что он обычно выбирает после последнего действия 

In [62]:
%%writefile sequence_agent.py
import random


sequence = [] # Последовательность действий противника
statistics = {}
# Структура для удобного извлечения наиболее вероятного след. действия
#например: {0: {0: 3, 1: 5, 2: 4}} что после 0 (камня) противник 3 раза ходил 0, 5 раз ходил 1
#и 4 раза ходил 2. При старте партии заполняется нулями
# {0: {0: 0, 1: 0, 2: 0}, 1: {0: 0, 1: 0, 2: 0}, 2: {0: 0, 1: 0, 2: 0}} """


def sequence_agent(observation, configuration):
    def reset_stat():
        # процедура обнуляет статистику
        global sequence
        global statistics
        sequence = []
        statistics = {}
        for i in range(0, configuration.signs):
            k = {}
            for j in range(0, configuration.signs):
                k[j] = 0
            statistics[i] = k

    def get_most_probable_next_action(last_action):
        # функция получает последнее действие оппонента,
        # и возвращает наиболее вероятное следующее
        # если статистика пустая, выбираем действие случайно
        
        d = statistics[last_action]
        act = None
        act_count = 0
        for k, v in d.items():
            if v > act_count:
                act = k
                act_count = v
                continue
        if act is None: act = random.randrange(0, configuration.signs)
        return act

    global sequence
    global statistics
    if observation.step == 0:
        reset_stat()
    # добавляем последнее действие в историю
    if observation.step > 0: sequence.append(observation.lastOpponentAction)
    
    if len(sequence) >= 2:
        # обновляем статистику и получаем наиболее подходящее действие
        statistics[sequence[-2]][sequence[-1]] += 1
        action = (get_most_probable_next_action(observation.lastOpponentAction) + 1) % configuration.signs
    else:
        action = random.randrange(0, configuration.signs)
    return action


Overwriting sequence_agent.py


После описания поведения агентов запустить турнир между ними и проверить, какая стратегия показывает себя лучше всех. Соревнование будем проводить по круговой системе (https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D1%83%D0%B3%D0%BE%D0%B2%D0%B0%D1%8F_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0)

In [63]:
agents = ['paper_agent.py',
          'scissors_agent.py',
          'random_agent.py',
          'next_agent.py',
          'simple_adaptation_agent.py',
          'sequence_agent.py'
         ]

agent_score = {'paper_agent.py': 0,
              'scissors_agent.py': 0,
              'random_agent.py': 0,
              'next_agent.py': 0,
              'simple_adaptation_agent.py': 0,
              'sequence_agent.py': 0
              }

In [66]:
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        player_1, player_2 = agents[i], agents[j]
        res = evaluate(
            "rps", #environment to use - no need to change
            [player_1, player_2], #agents to evaluate
            configuration={"episodeSteps": 100, "tieRewardThreshold":1 } #number of episodes
        )
        agent_score[player_1] += res[0][0]
        agent_score[player_2] += res[0][1]

raiting = {k: v for k, v in sorted(agent_score.items(), key=lambda item: item[1], reverse=True)}
for k, v in raiting.items():
    print(k, v)
        

sequence_agent.py 1025.0
simple_adaptation_agent.py 415.0
random_agent.py 18.0
scissors_agent.py -269.0
next_agent.py -296.0
paper_agent.py -893.0


как и следовало ижидать, победил самый умный

In [54]:
#evaluate(
#    "rps", #environment to use - no need to change
#    [player_1, player_2], #agents to evaluate
#    configuration={"episodeSteps": 100} #number of episodes 
#)

In [41]:
#env = make(
#    "rps",
#    configuration={
#        "episodeSteps": 50,
#        "tieRewardThreshold":1 
#    }
#)

In [53]:
#env.run(
#    ['paper_agent.py', 'random_agent.py']
#)
#env.render(mode="ipython", width=500, height=400)
#env.run
#env.render